<a href="https://colab.research.google.com/github/TranNhiem/Heuristic_attention_represenation_learning_ssl/blob/main/tensorflows/losses_optimizers/Self_Supervised_Loss_Testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Testing Contrastive Loss

In [ ]:
import tensorflow as tf
import numpy as np

## Initial_Random_Vector Representation

In [ ]:
## Example for Correct Concat the Tensor
t1 = [[1, 2, 3], [4, 5, 6]]
t2 = [[7, 8, 9], [10, 11, 12]]
t_12=tf.concat([t1, t2], 0)
tf.print(t_12)

In [ ]:
hidden_1= tf.random.uniform(
    shape=(32,512), minval=0, maxval=1, dtype=tf.dtypes.float32, seed=None, name=None)
hidden_2= tf.random.uniform(
    shape=(32,512), minval=0, maxval=1, dtype=tf.dtypes.float32, seed=None, name=None)
hidden_1_2=tf.concat((hidden_1, hidden_2),axis=-2)
hidden_1_2.shape
print("Batch_size_Hidden_rep1",hidden_1.shape)
print("Batch_size_Hidden_rep2",hidden_2.shape)
print("Double_Batch_size_Hidden_rep1_and_rep2",hidden_1_2.shape)

Batch_size_Hidden_rep1 (32, 512)
Batch_size_Hidden_rep2 (32, 512)
Double_Batch_size_Hidden_rep1_and_rep2 (64, 512)


## Contrastive Loss Version 1

In [ ]:
def get_negative_mask(batch_size):
    # return a mask that removes the similarity score of equal/similar images
    # Ensure distinct pair of image get their similarity scores
    # passed as negative examples
    batch_size= batch_size.numpy()
    negative_mask = np.ones((batch_size, 2 * batch_size), dtype=bool)
    for i in range(batch_size):
        negative_mask[i, i] = 0
        negative_mask[i, i+batch_size] = 0

    return tf.constant(negative_mask)

def nt_xent_asymetrize_loss_v2(p, z, temperature):  # negative_mask
    # L2 Norm
    batch_size = tf.shape(p)[0]
    batch_size=tf.cast(batch_size, tf.int32)
    labels = tf.one_hot(tf.range(batch_size), batch_size * 2)
    masks = tf.one_hot(tf.range(batch_size), batch_size)

    p_l2 = tf.math.l2_normalize(p, axis=1)
    z_l2 = tf.math.l2_normalize(z, axis=1)

    # Cosine Similarity distance loss

    # pos_loss = consie_sim_1d(p_l2, z_l2)
    pos_loss = tf.matmul(tf.expand_dims(p_l2, 1), tf.expand_dims(z_l2, 2))

    pos_loss = (tf.reshape(pos_loss, (batch_size, 1)))/temperature

    negatives = tf.concat([p_l2, z_l2], axis=0)
    # Mask out the positve mask from batch of Negative sample
    negative_mask = get_negative_mask(batch_size)
    print(negative_mask)
    
    loss = 0
    for positives in [p_l2, z_l2]:

        # negative_loss = cosine_sim_2d(positives, negatives)
        negative_loss = tf.tensordot(tf.expand_dims(
            positives, 1), tf.expand_dims(tf.transpose(negatives), 0), axes=2)
        l_labels = tf.zeros(batch_size, dtype=tf.float32)
        l_neg = tf.boolean_mask(negative_loss, negative_mask)

        l_neg = tf.reshape(l_neg, (batch_size, -1))
        l_neg /= temperature

        logits = tf.concat([pos_loss, l_neg], axis=1)  # [N, K+1]
        tf.keras.losses.SparseCategoricalCrossentropy()
        loss_ = tf.keras.losses.SparseCategoricalCrossentropy(
            from_logits=True, reduction=tf.keras.losses.Reduction.SUM)
        loss += loss_(y_pred=logits, y_true=l_labels)
    batch_size=tf.cast(batch_size, tf.float32)
    
    loss = loss/(2*batch_size)

    return loss

In [ ]:
loss=nt_xent_asymetrize_loss_v2(hidden_1, hidden_2, temperature=0.5)
tf.print(loss)

tf.Tensor(
[[False  True  True ...  True  True  True]
 [ True False  True ...  True  True  True]
 [ True  True False ...  True  True  True]
 ...
 [ True  True  True ... False  True  True]
 [ True  True  True ...  True False  True]
 [ True  True  True ...  True  True False]], shape=(32, 64), dtype=bool)
4.14318657


## Contrastive Loss Version 2

In [ ]:
def nt_xent_asymetrize_loss(z,  temperature):

    # Feeding data (ALready stack two version Augmented Image)[2*bs, 128]
    z = tf.math.l2_normalize(z, axis=1)

    similarity_matrix = tf.matmul(z, z, transpose_b=True)  # pairwise similarity
    similarity = tf.exp(similarity_matrix / temperature)

    ij_indices = tf.reshape(tf.range(z.shape[0]), shape=[-1, 2])
    ji_indices = tf.reverse(ij_indices, axis=[1])

    #[[0, 1], [1, 0], [2, 3], [3, 2], ...]
    positive_indices = tf.reshape(tf.concat(
        [ij_indices, ji_indices], axis=1), shape=[-1, 2])  # Indice positive pair
    # --> Output N-D array
    numerator = tf.gather_nd(similarity, positive_indices)
    # 2N-1 (sample)
    # mask that discards self-similarity
    negative_mask = 1 - tf.eye(z.shape[0])
    print(negative_mask)
    print(negative_mask.shape)

    # compute sume across dimensions of Tensor (Axis is important in this case)
    # None sum all element scalar, 0 sum all the row, 1 sum all column -->1D metric
    denominators = tf.reduce_sum(
        tf.multiply(negative_mask, similarity), axis=1)

    losses = -tf.math.log(numerator/denominators)
    return tf.reduce_mean(losses)

In [ ]:
loss= nt_xent_asymetrize_loss(hidden_1_2, temperature=0.5)
tf.print(loss)

tf.Tensor(
[[0. 1. 1. ... 1. 1. 1.]
 [1. 0. 1. ... 1. 1. 1.]
 [1. 1. 0. ... 1. 1. 1.]
 ...
 [1. 1. 1. ... 0. 1. 1.]
 [1. 1. 1. ... 1. 0. 1.]
 [1. 1. 1. ... 1. 1. 0.]], shape=(64, 64), dtype=float32)
(64, 64)
4.14759636


## Contrastive Loss Version 3 Original Simclr Loss

In [ ]:
def add_contrastive_loss(hidden,LARGE_NUM,
                         hidden_norm=True,
                         temperature=0.5,
                         weights=1.0, ):

  # Get (normalized) hidden1 and hidden2.
  if hidden_norm:
    hidden = tf.math.l2_normalize(hidden, -1)
  hidden1, hidden2 = tf.split(hidden, 2, 0)
  batch_size = tf.shape(hidden1)[0]
  # Gather hidden1/hidden2 across replicas and create local labels.

  hidden1_large = hidden1
  hidden2_large = hidden2
  labels = tf.one_hot(tf.range(batch_size), batch_size * 2)
  masks = tf.one_hot(tf.range(batch_size), batch_size)

  logits_aa = tf.matmul(hidden1, hidden1_large, transpose_b=True) / temperature
  logits_aa = logits_aa - masks * LARGE_NUM
  logits_bb = tf.matmul(hidden2, hidden2_large, transpose_b=True) / temperature
  logits_bb = logits_bb - masks * LARGE_NUM
  logits_ab = tf.matmul(hidden1, hidden2_large, transpose_b=True) / temperature
  logits_ba = tf.matmul(hidden2, hidden1_large, transpose_b=True) / temperature

  loss_a = tf.nn.softmax_cross_entropy_with_logits(
      labels, tf.concat([logits_ab, logits_aa], 1))
  loss_b = tf.nn.softmax_cross_entropy_with_logits(
      labels, tf.concat([logits_ba, logits_bb], 1))
  
  ## Original Simclr LOSS 
  loss = tf.reduce_mean(loss_a + loss_b)
  ## Sugest divide to 2 
  loss/=2
  return loss, logits_ab, labels

In [ ]:
LARGE_NUM=1e-9
loss, logits_ab, labels= add_contrastive_loss(hidden_1_2, LARGE_NUM=LARGE_NUM)

print("____________________")
print("this is loss value")
tf.print(loss)

print("____________________")
print("this is logit output")
tf.print(logits_ab)
print(logits_ab.shape)

print("____________________")
print("this is label positive ammong Negative batch")
tf.print(labels)
print(labels.shape)

____________________
this is loss value
4.16883945
____________________
this is logit output
[[1.4950105 1.56212342 1.51733971 ... 1.47297335 1.53903294 1.46565735]
 [1.47219634 1.51781917 1.46665359 ... 1.46836185 1.50715327 1.51138663]
 [1.44336891 1.48364687 1.46776772 ... 1.45604682 1.47696936 1.49555314]
 ...
 [1.5380224 1.50131226 1.51715374 ... 1.46726298 1.5188067 1.48607206]
 [1.51150751 1.48858261 1.48520684 ... 1.48390555 1.51678705 1.45680428]
 [1.47560918 1.55504358 1.49005628 ... 1.4609375 1.49416399 1.46781635]]
(32, 32)
____________________
this is label positive ammong Negative batch
[[1 0 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 [0 0 1 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
(32, 64)


In [ ]:
LARGE_NUM=1e-2
loss, logits_ab, labels= add_contrastive_loss(hidden_1_2, LARGE_NUM=LARGE_NUM)

print("____________________")
print("this is loss value")
tf.print(loss)

print("____________________")
print("this is logit output")
tf.print(logits_ab)
print(logits_ab.shape)

print("____________________")
print("this is label positive ammong Negative batch")
tf.print(labels)
print(labels.shape)

____________________
this is loss value
4.16858721
____________________
this is logit output
[[1.4950105 1.56212342 1.51733971 ... 1.47297335 1.53903294 1.46565735]
 [1.47219634 1.51781917 1.46665359 ... 1.46836185 1.50715327 1.51138663]
 [1.44336891 1.48364687 1.46776772 ... 1.45604682 1.47696936 1.49555314]
 ...
 [1.5380224 1.50131226 1.51715374 ... 1.46726298 1.5188067 1.48607206]
 [1.51150751 1.48858261 1.48520684 ... 1.48390555 1.51678705 1.45680428]
 [1.47560918 1.55504358 1.49005628 ... 1.4609375 1.49416399 1.46781635]]
(32, 32)
____________________
this is label positive ammong Negative batch
[[1 0 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 [0 0 1 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
(32, 64)


## Contrastive Loss Keras Version

In [ ]:
def nt_xent_symmetrize_keras(p, z, temperature):
    # cosine similarity the dot product of p,z two feature vectors
    x_i = tf.math.l2_normalize(p, axis=1)
    x_j = tf.math.l2_normalize(z, axis=1)
    similarity = (tf.matmul(x_i, x_j, transpose_b=True)/temperature)
    # the similarity from the same pair should be higher than other views
    batch_size = tf.shape(p)[0]  # Number Image within batch
    contrastive_labels = tf.range(batch_size)

    # Simlarilarity treat as logic input for Cross Entropy Loss
    # Why we need the Symmetrized version Here??
    loss_1_2 = tf.keras.losses.sparse_categorical_crossentropy(
        contrastive_labels, similarity, from_logits=True,)  # reduction=tf.keras.losses.Reduction.SUM
    loss_2_1 = tf.keras.losses.sparse_categorical_crossentropy(
        contrastive_labels, tf.transpose(similarity), from_logits=True, )
    loss=(loss_1_2 + loss_2_1) / 2
    avarage_loss= tf.reduce_mean(loss)
    return loss, avarage_loss

In [ ]:
loss,avarage_loss = nt_xent_symmetrize_keras(hidden_1, hidden_2, temperature=0.5)
print("____________________")
print("this is loss for each Image")
tf.print(loss)
loss.shape

print("____________________")
print("this is reduce_sum loss for each batch")
tf.print(avarage_loss)
avarage_loss.shape

____________________
this is loss for each Image
[3.47819 3.45527601 3.49630141 ... 3.4932723 3.45434523 3.49010658]
____________________
this is reduce_sum loss for each batch
3.46571207


TensorShape([])

# Binary Contrastive loss

In [ ]:
def binary_mask_nt_xent_asymetrize_loss(v1_object,v2_object,v1_background, v2_background, alpha = 1, temperature = 1):  # negative_mask
    # L2 Norm
    batch_size = tf.shape(v1_object)[0]
    v1_object = tf.math.l2_normalize(v1_object, -1)
    v2_object = tf.math.l2_normalize(v2_object, -1)
    v1_background = tf.math.l2_normalize(v1_background, -1)
    v2_background = tf.math.l2_normalize(v2_background, -1)

    INF = 1e9

    labels = tf.one_hot(tf.range(batch_size), batch_size * 2)
    masks = tf.one_hot(tf.range(batch_size), batch_size)

    #Object feature  dissimilar
    logits_o_aa = tf.matmul(v1_object, v1_object, transpose_b=True) / temperature
    logits_o_aa = logits_o_aa - masks * INF# remove the same samples
    logits_o_bb = tf.matmul(v2_object, v2_object, transpose_b=True) / temperature
    logits_o_bb = logits_o_bb - masks * INF# remove the same samples

    #bject and background feature  dissimilar
    logits_b_aa = tf.matmul(v1_object, v1_background, transpose_b=True) / temperature
    # logits_b_aa = logits_b_aa - masks * INF# remove the same samples
    logits_b_bb = tf.matmul(v2_object, v2_background, transpose_b=True) / temperature
    # logits_b_bb = logits_b_bb - masks * INF# remove the same samples

    #Object feature  similar
    logits_o_ab = tf.matmul(v1_object, v2_object, transpose_b=True) / temperature
    logits_o_ba = tf.matmul(v2_object, v1_object, transpose_b=True) / temperature
    #background feature  similar
    logits_b_ab = tf.matmul(v1_background, v2_background, transpose_b=True) / temperature
    logits_b_ba = tf.matmul(v2_background, v1_background, transpose_b=True) / temperature

    loss_a = tf.nn.softmax_cross_entropy_with_logits( labels, tf.concat([alpha * logits_o_ab + (1-alpha)*logits_b_ab, alpha * logits_o_aa + alpha * logits_b_aa], 1))
    loss_b = tf.nn.softmax_cross_entropy_with_logits( labels, tf.concat([alpha * logits_o_ba + (1-alpha)*logits_b_ba, alpha * logits_o_bb + alpha * logits_b_bb], 1))
    # loss_a = tf.nn.softmax_cross_entropy_with_logits(
    #     labels, tf.concat([logits_o_ab, logits_o_aa], 1))
    # loss_b = tf.nn.softmax_cross_entropy_with_logits(
    #     labels, tf.concat([logits_o_ba, logits_o_bb], 1))
    # print(loss_a)
    # print(loss_b)
    loss = tf.reduce_mean(loss_a + loss_b)
    loss = loss/2.0

    return loss

if __name__ == "__main__":
    v1_object = tf.random.uniform(shape = [32,256], maxval=1, dtype=tf.float32)
    v2_object = tf.random.uniform(shape = [32,256], maxval=1, dtype=tf.float32)
    v1_background = tf.random.uniform(shape = [32,256], maxval=1, dtype=tf.float32)
    v2_background = tf.random.uniform(shape = [32,256], maxval=1, dtype=tf.float32)
    loss = binary_mask_nt_xent_asymetrize_loss(v1_object,v2_object,v1_background, v2_background)

    print("loss : ",end = "")
    tf.print(loss)

loss : 4.57873726
